Install Dependency

In [55]:
%pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

In [2]:
client_id = "2a32ba42c2f346138baa11f711d69366"
client_secret = "e191f2f7a99f45fba77d6f193ca4c845"

In [3]:
scope = 'user-read-private user-read-email user-library-read user-library-modify playlist-modify-public playlist-modify-private playlist-read-private playlist-read-collaborative'

In [4]:
authentication = SpotifyOAuth(
        client_id=client_id, 
        client_secret=client_secret, 
        redirect_uri='http://localhost:8888/callback',
        scope=scope)

In [5]:
sp = spotipy.Spotify(auth_manager=authentication)

In [8]:
def user_saved_tracks():
    songs = []
    ids = []
    offset = 0

    while True:
        content = sp.current_user_saved_tracks(limit=50,offset=offset)
        songs += content['items']
        if content['next'] is not None:
            offset += 50
        else:
            break

    for i in songs:
        ids.append(i['track']['id'])

    index = 0

    audio_features = []

    while index < len(ids):
        audio_features += sp.audio_features(ids[index:index + 50])
        index += 50

    features_list = []
    for features in audio_features:
        features_list.append([features['energy'], features['liveness'],
                            features['tempo'], features['speechiness'],
                            features['acousticness'], features['instrumentalness'],
                            features['time_signature'], features['danceability'],
                            features['key'],features['loudness'], features['valence'],
                            features['mode'],features['uri']])
    
    df = pd.DataFrame(features_list, columns=['energy', 'liveness',
                                          'tempo', 'speechiness',
                                          'acousticness', 'instrumentalness',
                                          'time_signature', 'danceability',
                                          'key', 'loudness',
                                          'valence', 'mode', 'uri'])

    
    return df,ids

In [9]:
_user_saved_tracks,ids = user_saved_tracks()
_user_saved_tracks.head()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,loudness,valence,mode,uri
0,0.454,0.1360,90.038,0.1670,0.445,0.000000,4,0.709,9,-7.843,0.548,0,spotify:track:2tMoOS9grHtn4upzb4LwMG
1,0.739,0.1680,128.100,0.0314,0.687,0.000052,4,0.528,0,-6.320,0.542,0,spotify:track:3THVNKs3bdQekA02kK1hWD
2,0.906,0.0971,125.029,0.1800,0.697,0.004820,4,0.869,10,-7.386,0.864,0,spotify:track:6XLmRuk3wvOzJZzRToIvIN
3,0.633,0.1040,119.999,0.1930,0.395,0.000308,4,0.861,7,-6.702,0.810,1,spotify:track:70693JY8eUtdybBX30Imfb
4,0.424,0.1690,119.823,0.0359,0.844,0.003810,4,0.589,4,-12.006,0.561,0,spotify:track:7hMKAmUlUUYa5qEiHWK4bk


In [6]:
def get_songs_from_playlist(playlist_id):
    offset = 0
    songs = []
    ids = []
    while True:
        content = sp.playlist_tracks(playlist_id,limit=50, offset=offset)
        songs += content['items']
        if content['next'] is not None:
            offset += 50
        else:
            break

    for i in songs:
        ids.append(i['track']['id'])
    
    index = 0

    audio_features = []

    while index < len(ids):
        audio_features += sp.audio_features(ids[index:index + 50])
        index += 50

    features_list = []
    for features in audio_features:
        features_list.append([features['energy'], features['liveness'],
                            features['tempo'], features['speechiness'],
                            features['acousticness'], features['instrumentalness'],
                            features['time_signature'], features['danceability'],
                            features['key'],features['loudness'], features['valence'],
                            features['mode'],features['uri']])
    
    df = pd.DataFrame(features_list, columns=['energy', 'liveness',
                                          'tempo', 'speechiness',
                                          'acousticness', 'instrumentalness',
                                          'time_signature', 'danceability',
                                          'key', 'loudness',
                                          'valence', 'mode', 'uri'])
    return df,songs
    

In [7]:
data,songs = get_songs_from_playlist('https://open.spotify.com/playlist/09jMo9VfPVrSvVeLUYYkmr?si=ee10c7fd105e446e')
data.head()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,loudness,valence,mode,uri
0,0.448,0.106,95.050,0.0232,0.1630,0.000000,3,0.599,8,-6.312,0.168,1,spotify:track:0tgVpDi06FyKpA1z0VMD4v
1,0.780,0.081,124.949,0.1280,0.0622,0.000000,4,0.776,10,-4.374,0.666,0,spotify:track:0pqnGHJpmpxLKifKRmU6WP
2,0.541,0.130,124.959,0.0505,0.5680,0.000000,4,0.851,0,-6.825,0.448,1,spotify:track:7fa9MBXhVfQ8P8Df9OEbD8
3,0.524,0.111,95.010,0.0338,0.4140,0.000000,4,0.748,8,-5.599,0.661,1,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw
4,0.627,0.110,179.642,0.0476,0.0281,0.000008,4,0.468,6,-5.085,0.159,1,spotify:track:698ItKASDavgwZ3WjaWjtz


In [8]:
def get_songs(songs):
    list = []
    for i in range(len(songs)):
        list.append( songs[i]['track']['name'])

    return list

In [9]:
data['track_name'] = get_songs(songs)

In [10]:
data.head()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,loudness,valence,mode,uri,track_name
0,0.448,0.106,95.050,0.0232,0.1630,0.000000,3,0.599,8,-6.312,0.168,1,spotify:track:0tgVpDi06FyKpA1z0VMD4v,Perfect
1,0.780,0.081,124.949,0.1280,0.0622,0.000000,4,0.776,10,-4.374,0.666,0,spotify:track:0pqnGHJpmpxLKifKRmU6WP,Believer
2,0.541,0.130,124.959,0.0505,0.5680,0.000000,4,0.851,0,-6.825,0.448,1,spotify:track:7fa9MBXhVfQ8P8Df9OEbD8,Girls Like You (feat. Cardi B) - Cardi B Version
3,0.524,0.111,95.010,0.0338,0.4140,0.000000,4,0.748,8,-5.599,0.661,1,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,Closer
4,0.627,0.110,179.642,0.0476,0.0281,0.000008,4,0.468,6,-5.085,0.159,1,spotify:track:698ItKASDavgwZ3WjaWjtz,Faded


In [31]:
data.to_csv('Tamil_songs_spotify.csv')

In [11]:
data_copy = data.copy()

In [18]:
data.index= data_copy['track_name']

In [13]:
data.drop(['uri','track_name'], axis=1, inplace=True)

In [20]:
data.to_csv('spotify_english_songs.csv')

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import numpy as np

In [15]:
feature_set = data.columns.to_list()

In [16]:
scaler = MinMaxScaler().fit(data)
data = scaler.transform(data)
data = pd.DataFrame(data,columns= feature_set)

In [19]:
data.head()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,loudness,valence,mode
track_name,,,,,,,,,,,,
Perfect,0.454753,0.087413,0.269219,0.000000,0.164449,0.000000,0.50,0.523438,0.727273,0.830329,0.143100,1.0
Believer,0.795661,0.061428,0.449372,0.213095,0.062730,0.000000,0.75,0.753906,0.909091,0.891403,0.672549,0.0
Girls Like You (feat. Cardi B) - Cardi B Version,0.550248,0.112358,0.449432,0.055510,0.573143,0.000000,0.75,0.851562,0.000000,0.814162,0.440782,1.0
Closer,0.532792,0.092610,0.268978,0.021553,0.417738,0.000000,0.75,0.717448,0.727273,0.852798,0.667234,1.0
Faded,0.638555,0.091571,0.778917,0.049614,0.028319,0.000008,0.75,0.352865,0.545455,0.868997,0.133532,1.0


In [39]:
data.to_csv('scaled_spotify_tamil.csv')

In [16]:
_user_saved_tracks

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,loudness,valence,mode
0,0.066914,0.232371,0.126757,0.459459,0.504633,0.000000,1.000000,0.689320,0.818182,0.456369,0.385066,0.0
1,0.596654,0.315639,0.447479,0.012525,0.792112,0.000056,1.000000,0.396440,0.000000,0.623328,0.376281,0.0
2,0.907063,0.131148,0.421602,0.502307,0.803991,0.005251,1.000000,0.948220,0.909091,0.506468,0.847731,0.0
3,0.399628,0.149102,0.379217,0.545155,0.445236,0.000336,1.000000,0.935275,0.636364,0.581451,0.768668,1.0
4,0.011152,0.318241,0.377734,0.027357,0.978617,0.004150,1.000000,0.495146,0.363636,0.000000,0.404100,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
60,0.366171,0.036690,0.505764,0.363876,0.381088,0.000002,0.666667,0.695793,1.000000,0.477746,0.764275,0.0
61,0.514870,0.172521,0.446569,0.011866,0.596104,0.000676,1.000000,0.721683,0.090909,0.381715,0.459736,1.0
62,0.412639,0.341660,0.176818,0.101846,0.430981,0.000000,1.000000,0.661812,0.363636,0.606665,0.045388,0.0
63,0.633829,0.417122,0.557240,0.019776,0.267047,0.000031,1.000000,0.527508,0.909091,0.283271,0.756955,0.0


In [42]:
result = sp.current_user_recently_played(limit=1)
result['items'][0]['track']['name']

'Toshiba'

In [18]:
def get_cosim_artist_df(track_name, data, n):
    ss = StandardScaler()
    df_scaled = ss.fit_transform(data)
    data = pd.DataFrame(data=df_scaled, index=data.index)
    
    artist_array = np.array(data.T[track_name]).reshape(1,-1)
    dataset_array = data.drop(index=track_name).values
    
    cosim_scores = cosine_similarity(artist_array, dataset_array).flatten()
    track_names_array = data.drop(index=track_name).index.values
    
    df_result = pd.DataFrame(
        data = {
            'Song name'               : track_names_array,
            'cosim_' + track_name : cosim_scores,
        }
    )
    
    df_result = df_result.sort_values(by='cosim_' + track_name, ascending=False).head(n)    
    return df_result.reset_index(drop=True) 

In [21]:
get_cosim_artist_df('Aaluma Doluma', data, 10)

,Song name,cosim_Aaluma Doluma
0,"Kerala Song - From ""Natpe Thunai""",0.871842
1,Maane Maane,0.867788
2,Villadi Villan,0.867138
3,Kannum Kannumthaan,0.851623
4,Soora Thenga,0.832389
5,Dhoothu Varuma,0.830235
6,Unna Petha Aatha,0.794461
7,Where Is the Party,0.792186
8,Something Something,0.787325
9,Devuda Devuda,0.758900


In [76]:
sp.current_user()

{'country': 'IN',
 'display_name': 'A r u n',
 'email': '',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'external_urls': {'spotify': 'https://open.spotify.com/user/316n7rtosfjl4rljacpazoytwitq'},
 'followers': {'href': None, 'total': 4},
 'href': 'https://api.spotify.com/v1/users/316n7rtosfjl4rljacpazoytwitq',
 'id': '316n7rtosfjl4rljacpazoytwitq',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab6775700000ee85e9d66995e0ab1655db1a929c',
   'width': None}],
 'product': 'open',
 'type': 'user',
 'uri': 'spotify:user:316n7rtosfjl4rljacpazoytwitq'}

In [ ]:
def create_playlist(playlist_name):
    user_id = sp.current_user()
    sp.user_playlist_create(user_id['id'], playlist_name, public=True)
    sp.playlist_add_items('0ZAgEmVsa8ytsePvrrk1Du',ids)
    

In [43]:
sp.current_user_top_artists(limit = 10)

{'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6AiX12wXdXFoGJ2vk8zBjy'},
   'followers': {'href': None, 'total': 4708635},
   'genres': ['filmi', 'kollywood', 'tamil pop'],
   'href': 'https://api.spotify.com/v1/artists/6AiX12wXdXFoGJ2vk8zBjy',
   'id': '6AiX12wXdXFoGJ2vk8zBjy',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5eb7249dd34afa1c09e205260f4',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab676161000051747249dd34afa1c09e205260f4',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f1787249dd34afa1c09e205260f4',
     'width': 160}],
   'name': 'Yuvan Shankar Raja',
   'popularity': 75,
   'type': 'artist',
   'uri': 'spotify:artist:6AiX12wXdXFoGJ2vk8zBjy'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/4zCH9qm4R2DADamUHMCa6O'},
   'followers': {'href': None, 'total': 7850798},
   'genres': ['desi hip hop', 'filmi', 'tamil hip hop'

In [44]:
sp.current_user_top_tracks(limit = 10)

{'items': [{'album': {'album_type': 'SINGLE',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1QHr8p3h0ayWLaaYK0X6bi'},
      'href': 'https://api.spotify.com/v1/artists/1QHr8p3h0ayWLaaYK0X6bi',
      'id': '1QHr8p3h0ayWLaaYK0X6bi',
      'name': 'Dhibu Ninan Thomas',
      'type': 'artist',
      'uri': 'spotify:artist:1QHr8p3h0ayWLaaYK0X6bi'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/0nMjhemqRwrboQGcs92fh2'},
      'href': 'https://api.spotify.com/v1/artists/0nMjhemqRwrboQGcs92fh2',
      'id': '0nMjhemqRwrboQGcs92fh2',
      'name': 'Kapil Kapilan',
      'type': 'artist',
      'uri': 'spotify:artist:0nMjhemqRwrboQGcs92fh2'}],
    'available_markets': ['AD',
     'AE',
     'AR',
     'AT',
     'AU',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DE',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'F

In [8]:
result = sp.search(q='Aaluma Doluma', type='track')
result

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Aaluma+Doluma&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4zCH9qm4R2DADamUHMCa6O'},
       'href': 'https://api.spotify.com/v1/artists/4zCH9qm4R2DADamUHMCa6O',
       'id': '4zCH9qm4R2DADamUHMCa6O',
       'name': 'Anirudh Ravichander',
       'type': 'artist',
       'uri': 'spotify:artist:4zCH9qm4R2DADamUHMCa6O'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
     

In [10]:
result['tracks']['items'][0]['uri']

'spotify:track:1Do2hDE0etMakAEQbyOd4L'